### First We start with the imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
#we will use pytorch for the neural network here
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



### Image Preprocessing

In [ ]:
#now will process the image to fit the model specs
transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

#load the data 
#I'll leave it as a placeholder for now until Ahmed provides the data

In [ ]:
model = models.resnet50(pretrained=True)
